In [1]:
import os
import cv2
import face_recognition
import face_recognition as fr
import numpy as np

In [ ]:
cam = cv2.VideoCapture(0)
img_couter = 0

path = './faces'
while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame")
        break
    cv2.imshow("test", frame)
    k = cv2.waitKey(1)
    if k % 256 == 27:
        print("Escape Hit, closing the application")
        break
    elif k%256 == 32:
        img_name = input("Enter the name of the image: ")
        cv2.imwrite(os.path.join(path , '{}.jpg'.format(img_name)),frame)
        print("Image Captured")
        img_couter += 1
cam.release()
cv2.destroyAllWindows()

In [2]:
# encoding all the faces using get_encoded_faces () function
def get_encoded_faces():
    encoded = {}
    for dirpath, dnames, fnames in os.walk("./faces"):
        for f in fnames:
            if f.endswith(".jpg") or f.endswith(".png"):
                face = fr.load_image_file("faces/" + f)
                encoding = fr.face_encodings(face)[0]
                encoded[f.split(".")[0]] = encoding
    return encoded

In [3]:
# encode a face given the file name
def unknown_image_encoded(img):
    face = fr.load_image_file("faces/" + img)
    encoding = fr.face_encodings(face)[0]
    return encoding

In [4]:
# finding all the faces images and label that images
def classify_face(im):
    faces = get_encoded_faces()
    faces_encoded = list(faces.values())
    known_face_names = list(faces.keys())
    img = cv2.imread(im, 1)
    face_locations = face_recognition.face_locations(img)
    unknown_face_encodings = face_recognition.face_encodings(img, face_locations)

    face_names = []
    for face_encoding in unknown_face_encodings:
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(faces_encoded, face_encoding)
        name = "Unknown"
        # use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(faces_encoded, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        face_names.append(name)
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Draw a box around the face
            cv2.rectangle(img, (left - 20, top - 20), (right + 20, bottom + 20), (255, 0, 0), 2)
            # Draw a label with a name below the face
            cv2.rectangle(img, (left - 20, bottom - 15), (right + 20, bottom + 20), (255, 0, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(img, name, (left - 20, bottom + 15), font, 1.0, (255, 255, 255), 2)
    # Display the resulting image
    while True:
        cv2.imshow('Face Detection', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            return face_names

In [5]:
print(classify_face("./daju_bahini.jpg"))

['dai', 'bahini']
